In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import PIL 
import os
import cv2
from PIL import Image 
from numpy import asarray 

In [2]:
directory  = "./DataSet"

In [3]:
# To Get the first word in the filepath

def parse_filepath(filepath):
    path, filename = os.path.split(filepath)
    filename, ext = os.path.splitext(filename)
    label = filename.split("_")
    #print(label)
    return label

In [4]:
# Putting the Images into a dataframe so that it can be passed easily.
# Here the images filepath is put along with that of the labels.
import glob


files = glob.glob(os.path.join(directory,"*.jpg"))
# print(files)
labels = list(files)
labels = list(map(parse_filepath,files))
# print(labels)
df = pd.DataFrame(labels)
df['filepath'] = files
df.columns = ['labels','filepath']
df.dropna()
df.head()
# df.describe()

,labels,filepath
0,2a6y6,./DataSet/2a6y6.jpg
1,2a2yx,./DataSet/2a2yx.jpg
2,2a5pf,./DataSet/2a5pf.jpg
3,2a5cn,./DataSet/2a5cn.jpg
4,2a6yy,./DataSet/2a6yy.jpg


In [5]:
# To get the size of one image file 

def img_size():
    x =files[0]
    image = Image.open(x)
    print(image.mode)
    return (image.size)   
        

In [6]:
img_size()

RGB


(160, 80)

In [13]:
from tensorflow.keras.utils import to_categorical
from PIL import Image

def pass_Data(df,indices,training_bool,batch_size=16):
    images, labels_1 = [], []
    while True:
        for i in indices:
            r = df.iloc[i]
            file, label = r['filepath'], r['labels']
            im = Image.open(file)
            im = np.array(im) / 255.0
            im.resize(277,277,3)
            #print(im.length)
            images.append(np.array(im))
            labels_1.append(np.array([np.array(to_cate(i, 36)) for i in label]))
            if len(images) >= batch_size:
                yield np.array(images), np.array(labels_1)
                images, labels_1 = [], []
        if not training_bool:
            break

In [14]:
# Splitting the data set into test train and validation set

p = np.random.permutation(len(df)) # random images are distributed.
train_up_to = int(len(df) * 0.8) # the train and the test set is divided into 80:20 
train_idx = p[:train_up_to]
test_idx = p[train_up_to:]

# split train_idx further into training and validation set
train_up_to = int(train_up_to * 0.8) # the train set is further divided into the validation and the training data set
train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]

print('train count: %s, valid count: %s, test count: %s' % (
    len(train_idx), len(valid_idx), len(test_idx)))

train count: 5, valid count: 2, test count: 2


In [15]:
#  Generating the test and the validation data 

batch_size = 2# insert Number
valid_batch_size = 1 #inset Number 

# def pass_Data(df,indices,training_bool,batch_size=16)
train_gen = pass_Data(df,train_idx,True,batch_size = batch_size)
validation_gen = pass_Data(df,valid_idx,True,batch_size = valid_batch_size)


In [16]:
# To calculate the validation and the training steps

validation_steps = (len(valid_idx)//valid_batch_size)
train_steps = (len(train_idx)//batch_size)
print(validation_steps,train_steps)

2 2


In [17]:
#  Alex_net Model making
def Alex_net():
#     THe input shape is always 257*257*3 or 256*256*3 and the images should be reshaped into it, if it is not that currently
    image_shape = (227,227,3)
    np.random.seed(1000)
    model = Sequential()
#     First Layer which is the convolution Layer
    model.add(Conv2D(filters = 96,input_shape = image_shape,kernel_size=(11,11),strides=(4,4),padding = 'valid'))
    model.add(Activation('relu'))
#     Next is the Pooling layer
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding = 'valid'))

#     The first convolution and max pooling layer is done .

#     Second Layer 
    model.add(Conv2D(filters = 256,kernel_size=(5,5),strides=(1,1),padding = 'valid'))
    model.add(Activation('relu'))
#     Next is the Pooling layer
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding = 'valid'))
    
#     Second Layer Done 

#     Third Layer is only the convolution layer. There is no maxPooling layer at all

    model.add(Conv2D(filters = 384,kernel_size=(3,3),strides=(1,1),padding = 'valid'))
    model.add(Activation('relu'))

# Fourth layer- only convolution layer
    
    model.add(Conv2D(filters = 384,kernel_size=(3,3),strides=(1,1),padding = 'valid'))
    model.add(Activation('relu'))
    
# Fifth layer- one convolution and one pooling layer
    model.add(Conv2D(filters = 256,kernel_size=(3,3),strides=(1,1),padding = 'valid'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding = 'valid'))

        
    model.add(Flatten())
# Sixth fully connected layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
#Seventh fully connected layer
    model.add(Flatten())
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
#Final softmax layer
    model.add(Dense(5*36))
    model.add(Activation('softmax'))
    #model.add(Reshape(5,36))
    
    model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
    history = model.fit(train_gen,steps_per_epoch= train_steps,epochs = 2,validation_data = validation_gen, validation_steps = validation_steps)
    return history

In [18]:
Alex_net()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_8 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_9 (Activation)    (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 9, 9, 384)        

ValueError: invalid literal for int() with base 10: 'a'